<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/1_pubSub_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-cloud-pubsub

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

In [3]:
import fsspec
import gcsfs

Produtor

In [ ]:
import csv
import time
from google.cloud import pubsub_v1
import os
from google.cloud import storage
import pandas as pd

service_account_key = r"/content/soulcode-projeto-final-4b88bea6e07a.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

topico = 'projects/soulcode-projeto-final/topics/ingestor_dados'
publisher = pubsub_v1.PublisherClient()


entrada = r"/content/2_temp_temp_pandas_total_pop_ano_uf.csv"

with open(entrada, 'rb') as file:
    for row in file:
        print('Publicando no topico: ', topico)
        publisher.publish(topico,row)
        time.sleep(2)

Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados
Publicando no topico:  projects/soulcode-projeto-final/topics/ingestor_dados

Consumidor

In [ ]:
import csv
import time
from google.cloud import pubsub_v1
import os


service_account_key = r'/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

subscription = 'projects/soulcode-projeto-final/subscriptions/consumidor_dados_violencia'
subscriber = pubsub_v1.SubscriberClient()

def monstrar_msg(mensagem):
  print(('Mensagem: {}'.format(mensagem)))
  mensagem.ack()

subscriber.subscribe(subscription,callback=monstrar_msg)

while True:
  time.sleep(3)

Mensagem: Message {
  data: b'1,AL,3358963,2016\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'5,CE,9187103,2020\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'21,RR,631181,2020\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'0,AC,906876,2021\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'10,MG,20997560,2016\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'20,RO,1787279,2016\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'23,SC,6910553,2016\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'3,AP,797722,2017\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'6,DF,3039444,2017\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'21,RR,522636,2017\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message {
  data: b'22,RS,11322895,2017\n'
  ordering_key: ''
  attributes: {}
}
Mensagem: Message

KeyboardInterrupt: ignored

Template Stream

In [ ]:
pip install apache_beam[gcp]

In [2]:
import apache_beam as beam
from apache_beam import window
p1 = beam.Pipeline()

class separar_linhas(beam.DoFn):
  def process(self,record):
    return [record.decode("utf-8").split(',')]

mkt = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/2_temp_temp_pandas_total_pop_ano_uf.csv', skip_header_lines=0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    | "Window" >> beam.WindowInto(window.SlidingWindows(10,5))
    |'Saida de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('resultado.txt')
)
p1.run()

['', 'UF', 'populacao_estimada', 'Ano']
['0', 'AC', '816687', '2016']
['1', 'AL', '3358963', '2016']
['2', 'AM', '4001667', '2016']
['3', 'AP', '782295', '2016']
['4', 'BA', '15276566', '2016']
['5', 'CE', '8963663', '2016']
['6', 'DF', '2977216', '2016']
['7', 'ES', '3973697', '2016']
['8', 'GO', '6695855', '2016']
['9', 'MA', '6954036', '2016']
['10', 'MG', '20997560', '2016']
['11', 'MS', '2682386', '2016']
['12', 'MT', '3305531', '2016']
['13', 'PA', '8305359', '2016']
['14', 'PB', '3999415', '2016']
['15', 'PE', '9410336', '2016']
['16', 'PI', '3212180', '2016']
['17', 'PR', '11242720', '2016']
['18', 'RJ', '16635996', '2016']
['19', 'RN', '3474998', '2016']
['20', 'RO', '1787279', '2016']
['21', 'RR', '514229', '2016']
['22', 'RS', '11286500', '2016']
['23', 'SC', '6910553', '2016']
['24', 'SE', '2265779', '2016']
['25', 'SP', '44749699', '2016']
['26', 'TO', '1532902', '2016']
['0', 'AC', '829619', '2017']
['1', 'AL', '3375823', '2017']
['2', 'AM', '4063614', '2017']
['3', 'AP',

In [1]:
import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam import window

pipeline_options = {
    'project': 'soulcode-projeto-final' ,
    'runner': 'DataflowRunner',
    'region': 'southamerica-east1',
    'staging_location': 'gs://data_lake_ingest_data/temporario',
    'temp_location': 'gs://data_lake_ingest_data/temporario',
    'template_location': 'gs://data_lake_ingest_data/template/data_streaming',
    'save_main_session': True ,
    'streaming' : True }

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)
p1 = beam.Pipeline(options=pipeline_options)

serviceAccount = r'/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

subscription = 'projects/soulcode-projeto-final/subscriptions/consumidor_dados_violencia'
saida = 'projects/soulcode-projeto-final/topics/ingestor_dados_saida'

class separar_linhas(beam.DoFn):
  def process(self,record):
    return [record.decode("utf-8").split(',')]

#class filtro(beam.DoFn):
 # def process(self,record):
  #  if int(record[8]) > 0:
   #   return [record]


data = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/2_temp_temp_pandas_total_pop_ano_uf.csv', skip_header_lines=0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    | "Window" >> beam.WindowInto(window.SlidingWindows(10,5))
    #|'Saida de dados' >> beam.Map(print)
    | "Escrever no Tópico" >> beam.io.WriteToPubSub(saida)
    #|'Gravar resultado' >> beam.io.WriteToText('resultado.txt')
)

result = p1.run()
result.wait_until_finish()

'UNKNOWN'